# Importing libraries

In [1]:
IS_EXPERIMENT = True

if IS_EXPERIMENT:
    
    from comet_ml import Experiment
    experiment = Experiment(
        api_key="VCyOxE7IjP9abOTTSwvn2gfs4",
        project_name="pointcloudfcd",
        workspace="mrugnivenko",
    )

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/mrugnivenko/pointcloudfcd/e90b1cf5fabc46c0ae5440cb91703f08



In [2]:
import os
import imp
import time
import utils
import pickle
import datetime
import numpy as np
from tqdm import tqdm
from utils.fcd import *
from utils.data_processor import *
from utils.crop import get_loader_crop
from sklearn.model_selection import KFold

import warnings
warnings.filterwarnings("ignore")

%config Completer.use_jedi = False

# Specifying parameters of model

In [3]:
"""
There 3 types of model training:
- whole: on the whole brain 
- temple: on the temple part of the brain
- nottemple: on the whole brain without temple part 
"""
BRAIN_TYPE = 'nottemple' # 'whole', 'temple', 'nottemple'

"""
There 2 types of Local Agragation Operators:
- pospoolxyz
- pointwisemlp
"""
CFG = 'cfgs/brain/brain_pospoolxyz.yaml' # 'cfgs/brain/brain_pointwisemlp.yaml'

"""
There 3 GPUs:
- 0
- 1
- 2
"""
# DEVICE = 0
# torch.cuda.set_device(f"cuda:{DEVICE}")

BATCH_SIZE = 16
CROP_SIZE = 64

"""
Whether to use folded training
"""
FOLDED = True

"""
Whether to use absolute coordinates of points or relative ones
"""
IS_RETURN_ABS_COORDS = True

"""
Whether to use only meaningful points, which are not air-points
"""
IS_RETURN_PC_WITHOUT_AIR_POINTS = False

"""

"""
COIN_FLIP_THRESHOLD = 0.8


WEIGHTED_LOSS = True
LOG_CONFUSION_MATRIX = True
FINE_TUNE = False
EPOCHS = 400

#For early stopping
PATIENCE = 150
 #NNN
EXP_NAME = f'{str(datetime.date.today())}_{BATCH_SIZE}bs_{"abs" if IS_RETURN_ABS_COORDS else "rel"}coords_{"no" if IS_RETURN_PC_WITHOUT_AIR_POINTS else ""}air_cropsize{CROP_SIZE}_epochs{EPOCHS}_{BRAIN_TYPE}'

# For fine-tuning
PRETRAINED_MODEL_PATH = 'checkpoints/3.pth'

#For loss
LOSS_TYPE = 'BCE'

#lala
RES_PATH = 'predictions/' + EXP_NAME
IS_CROP = True
LOGGING_PERIOD = 10
print(EXP_NAME)

2021-11-08_16bs_abscoords_air_cropsize64_epochs400_nottemple


In [4]:
torch.cuda.set_device("cuda:1") # DO NOT CHANGE THIS
config = config_seting(CFG)

# Data loading

In [5]:
allowed_subjects = np.load('raw_data/sub_with_all_data.npy', allow_pickle=True).tolist()

if not allowed_subjects:
    allowed_subdirs = [f"sub-{name[:-4]}" for name in os.listdir(f"raw_data/normalized_label")]
else:
    allowed_subdirs = [f'sub-{name}' for name in allowed_subjects]
    
brains = [[f"raw_data/output/fmriprep/{subdir}/anat/{name}" for name in os.listdir(f"raw_data/output/fmriprep/{subdir}/anat") if name.endswith("Asym_desc-preproc_T1w.nii.gz")] for subdir in os.listdir(f"raw_data/output/fmriprep/") if subdir.count(".") == 0 and subdir.startswith("sub") and subdir in allowed_subdirs]
brains = [item for sublist in brains for item in sublist]
brain_nums = [x.split('/')[3][4:] for x in brains]

if BRAIN_TYPE == 'whole':
    BRAIN_TYPE = 'full'
    
brains_loaded = [f"dataset_ready_to_use/data_only_usefull_areas/{BRAIN_TYPE}/brains/{num}.npy" for num in tqdm(brain_nums)]
labels_loaded = [f"dataset_ready_to_use/data_only_usefull_areas/{BRAIN_TYPE}/labels/{num}.npy" for num in tqdm(brain_nums)]
curv_loaded = [f"dataset_ready_to_use/data_only_usefull_areas/{BRAIN_TYPE}/curv/{num}.npy" for num in tqdm(brain_nums)]
thickness_loaded = [f"dataset_ready_to_use/data_only_usefull_areas/{BRAIN_TYPE}/thickness/{num}.npy" for num in tqdm(brain_nums)]
sulc_loaded = [f"dataset_ready_to_use/data_only_usefull_areas/{BRAIN_TYPE}/sulc/{num}.npy" for num in tqdm(brain_nums)]

100%|██████████| 81/81 [00:00<00:00, 431140.39it/s]


# Training and validation

In [ ]:
def log_metrics(experiment, metrics_names, metrics_values, epoch):
    for name, value in zip(metrics_names, metrics_values):
        experiment.log_metric(name, value, epoch=epoch)
        
kf = KFold(n_splits=5, shuffle=True, random_state=42)
pickle.dump(kf, open(f"predictions/KFold/{EXP_NAME}.pkl", "wb"))

for e, (train_idxs, test_idxs) in tqdm(enumerate(kf.split(brains_loaded))):  
    
    train_dict = {
        'brains': [brains_loaded[idx] for idx in train_idxs],
        'labels': [labels_loaded[idx] for idx in train_idxs],
#         'curvs': [curv_loaded[idx] for idx in train_idxs],
#         'thickness': [thickness_loaded[idx] for idx in train_idxs],
#         'sulc': [sulc_loaded[idx] for idx in train_idxs]
    }
    
    test_dict = {
        'brains': [brains_loaded[idx] for idx in test_idxs],
        'labels': [labels_loaded[idx] for idx in test_idxs],
#         'curvs': [curv_loaded[idx] for idx in test_idxs],
#         'thickness': [thickness_loaded[idx] for idx in test_idxs],
#         'sulc': [sulc_loaded[idx] for idx in test_idxs]
    }
    
    out = get_loader_crop(config=config,
                          batch_size=BATCH_SIZE,
                          num_points=config.num_points,
                          train_dict=train_dict,
                          test_dict=test_dict,
                          crop_size=CROP_SIZE,
                          is_folded=FOLDED,
                          return_abs_coords=IS_RETURN_ABS_COORDS,
                          return_pc_without_air_points=IS_RETURN_PC_WITHOUT_AIR_POINTS,
                          coin_flip_threshold = COIN_FLIP_THRESHOLD,
                          weighted_loss = WEIGHTED_LOSS
                         )
    
    if WEIGHTED_LOSS:
        train_loader, test_loader, weights = out
    else:
        train_loader, test_loader = out
        weights = None
        
    print(f"size of train dataset: {len(train_loader.dataset)}")
    print(f"size of test dataset: {len(test_loader.dataset)}")
        
    model, criterion = build_multi_part_segmentation(config=config,
                                                     weights=weights,
                                                     type=LOSS_TYPE,
                                                    )
    if FINE_TUNE:
        model.load_state_dict(torch.load(PRETRAINED_MODEL_PATH))
    model.cuda()
    criterion.cuda()
        
    optimizers = {"adam":torch.optim.Adam(model.parameters(),
                                     lr=config.base_learning_rate,
                                     weight_decay=config.weight_decay),
                 "adamW":torch.optim.AdamW(model.parameters(),
                                      lr=config.base_learning_rate,
                                      weight_decay=config.weight_decay)}
    
    if config.optimizer in optimizers:
        optimizer=optimizers[config.optimizer]
    else:
        raise NotImplementedError(f"Optimizer {config.optimizer} not supported")
        
    scheduler = get_scheduler(optimizer, len(train_loader), config)
    
    for epoch in tqdm(range(1, EPOCHS + 1)):
        
        loss, opt, roc = train(epoch,
                               train_loader,
                               model, criterion,
                               optimizer,
                               scheduler,
                               config
                              )
        
        if IS_EXPERIMENT:
            log_metrics(experiment, 
                        [f"train AUC-ROC (fold #{e + 1}))", f"learning rate (fold #{e + 1}))", f"train loss (fold #{e + 1}))"], 
                        [roc, optimizer.param_groups[0]['lr'], loss],
                        epoch
                       )
            
        if epoch == 1 or epoch % LOGGING_PERIOD == 0:
            metrics_dict, confusion_matrix = validate(epoch,
                                                      test_loader,
                                                      model,
                                                      criterion,
                                                      config,
                                                      num_votes=1, 
                                                      log_confusion_matrix=LOG_CONFUSION_MATRIX,
                                                      is_crop=IS_CROP
                                                     )
            
            if IS_EXPERIMENT:   
                for name, value in metrics_dict.items():
                    experiment.log_metric(f"test {name} (fold #{e + 1}))", value, epoch=epoch)
                if LOG_CONFUSION_MATRIX:
                    experiment.log_confusion_matrix(title=f"Test confusion epoch={epoch} (fold #{e + 1}))", 
                                                    matrix=confusion_matrix, labels=['No FCD', 'FCD'])
                    

    path = f'../PointCloudResNet/checkpoints/{EXP_NAME}_{e + 1}_fold.pth'
    torch.save(model.state_dict(), path)  
    
    del model, train_loader, test_loader

0it [00:00, ?it/s]

Weights are: [0.0064849853515625, 0.9935150146484375]
Train dataset created
Test dataset created
size of train dataset: 64
size of test dataset: 17



100%|██████████| 400/400 [1:37:37<00:00, 14.64s/it]
1it [1:41:32, 6093.00s/it]